In [18]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from langchain_community.utilities import GoogleSerperAPIWrapper
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat

from autogen_ext.tools.http import HttpTool
from dotenv import load_dotenv
from autogen_agentchat.ui import Console
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [19]:
planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

In [20]:
search_tool_wrapper = GoogleSerperAPIWrapper(type='search')

def search_web(query:str) ->str:
    """Search the web for the given query and return the results."""
    try:
        results = search_tool_wrapper.run(query)
        return results
    except Exception as e:
        print(f"Error occurred while searching the web: {e}")
        return "No results found." 
    

web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web],
    reflect_on_tool_use=True,
    system_message='''
        You are a web search agent.
        Your only tool is search_web - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [21]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100

data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided (percentage_change_tool).

        If you have not seen the data, ask for it.

    ''',
)

text_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=20)
combined_termination = text_mention_termination | max_message_termination



In [22]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
Only select one agent.
'''

selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To solve this task, we need to take the following steps:

1. Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. Find the total rebounds for this player in both the 2007-2008 and 2008-2009 seasons.
3. Calculate the percentage change in the player's total rebounds between these two seasons.

I will delegate these tasks as follows:

1. WebSearchAgent: Search for the player on the Miami Heat with the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Find the total rebounds for this player in the 2007-2008 season.
3. WebSearchAgent: Find the total rebounds for this player in the 2008-2009 season.
4. DataAnalystAgent: Calculate the percentage change in total rebounds between the 20

TaskResult(messages=[TextMessage(id='cccddbe8-e197-4b99-a923-3b3471c4226b', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 21, 10, 39, 6, 519073, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='43c78194-2e3b-4aa1-baba-094760b25fc6', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=197), metadata={}, created_at=datetime.datetime(2025, 7, 21, 10, 39, 10, 920628, tzinfo=datetime.timezone.utc), content="To solve this task, we need to take the following steps:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 season.\n2. Find the total rebounds for this player in both the 2007-2008 and 2008-2009 seasons.\n3. Calculate the percentage change in the player's total rebounds between the

In [25]:
await selector_team.reset()

In [24]:
# If Data analyst agent is not selected after web search agent
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence

def my_selector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):

    if messages[-1].source != web_search_agent.name:
        return data_analyst_agent.name
    return None


selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=False,
    selector_func=my_selector_fun)


await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (DataAnalystAgent) ----------
I currently do not have access to specific datasets, such as player statistics for the Miami Heat for the 2006-2007 or 2007-2008, 2008-2009 seasons. Please provide the data on the player's total rebounds for each of those seasons, and I can calculate the percentage change for you.
---------- TextMessage (DataAnalystAgent) ----------
If you have the data on the player's total rebounds for each of the relevant seasons, please provide it so I can proceed with the analysis.
---------- TextMessage (DataAnalystAgent) ----------
Please provide the data on the Miami Heat player's total rebounds for the 2007-2008 and 2008-2009 seasons, so I can calculate the percentage change for you.
---------- TextMessage (DataAnalystAgent)

TaskResult(messages=[TextMessage(id='1825f21c-dde3-46ee-85c7-24ea6bd45772', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 21, 10, 39, 25, 813066, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='a3dc934c-f1e3-4e62-b10c-5e425f310519', source='DataAnalystAgent', models_usage=RequestUsage(prompt_tokens=150, completion_tokens=68), metadata={}, created_at=datetime.datetime(2025, 7, 21, 10, 39, 28, 51584, tzinfo=datetime.timezone.utc), content="I currently do not have access to specific datasets, such as player statistics for the Miami Heat for the 2006-2007 or 2007-2008, 2008-2009 seasons. Please provide the data on the player's total rebounds for each of those seasons, and I can calculate the percentage change for you.", type='TextMessage'), Tex